In [1]:
pip install nltk


In [2]:
import nltk
nltk.download('treebank')
nltk.download('universal_tagset')


[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

Dưới đây là nội dung được viết lại để dễ dàng chèn vào tệp Jupyter Notebook (`.ipynb`) với định dạng Markdown:

```markdown
# Bộ dữ liệu Treebank

Bộ dữ liệu Treebank là một tập hợp dữ liệu ngôn ngữ được chú thích thủ công, chủ yếu được sử dụng trong xử lý ngôn ngữ tự nhiên (NLP) và học máy. Nó bao gồm các câu với nhãn từ loại (POS tags), cú pháp, và đôi khi là ngữ nghĩa. Dưới đây là một số chi tiết về bộ dữ liệu Treebank mà chúng ta sử dụng từ thư viện NLTK:

## Thành phần của Dữ liệu Treebank

### Câu (Sentences)
- Các câu trong Treebank được lấy từ nhiều nguồn văn bản khác nhau, như báo chí, sách, và các văn bản khác.
- Mỗi câu được chú thích thủ công để đảm bảo độ chính xác cao.

### Nhãn Từ Loại (POS Tags)
- Mỗi từ trong câu được gán một nhãn từ loại tương ứng, chẳng hạn như danh từ (N), động từ (V), tính từ (Adj), trạng từ (Adv), v.v.
- Treebank sử dụng nhiều bộ nhãn từ loại khác nhau; trong NLTK, bạn có thể chọn bộ nhãn "universal" để có một tập nhãn đơn giản hơn.

### Cú pháp (Syntax)
- Treebank cũng bao gồm thông tin về cấu trúc cú pháp của câu, chẳng hạn như cây phân tích cú pháp (parse tree), giúp xác định cấu trúc phân đoạn của câu.

### Ngữ nghĩa (Semantics)
- Một số phiên bản của Treebank cũng bao gồm chú thích về ngữ nghĩa, mặc dù thông tin này ít phổ biến hơn so với thông tin về nhãn từ loại và cú pháp.

## Ví dụ về Cấu Trúc của Dữ liệu

Dưới đây là một ví dụ về cách một câu trong Treebank có thể được chú thích:

**Câu gốc**: "The quick brown fox jumps over the lazy dog."

**Nhãn từ loại**:
- "The" -> Determiner (DET)
- "quick" -> Adjective (ADJ)
- "brown" -> Adjective (ADJ)
- "fox" -> Noun (NOUN)
- "jumps" -> Verb (VERB)
- "over" -> Preposition (ADP)
- "the" -> Determiner (DET)
- "lazy" -> Adjective (ADJ)
- "dog" -> Noun (NOUN)
```

Bạn có thể chèn đoạn Markdown này vào tệp Jupyter Notebook của mình để cung cấp thông tin chi tiết về bộ dữ liệu Treebank. Nội dung này giải thích các thành phần của dữ liệu và đưa ra ví dụ cụ thể về cách dữ liệu được chú thích.

1. Khởi tạo: Tải và chuẩn bị dữ liệu từ bộ dữ liệu Treebank.
2. Xây dựng xác suất:
- Start probabilities: Tính xác suất khởi đầu cho mỗi nhãn từ loại.
- Transition probabilities: Tính xác suất chuyển tiếp giữa các nhãn từ loại.
- Emission probabilities: Tính xác suất phát âm của mỗi từ cho mỗi nhãn từ loại.
3. Viterbi Algorithm: Triển khai giải thuật Viterbi để tìm chuỗi nhãn từ loại có xác suất cao nhất cho mỗi câu trong tập kiểm thử.
4. Kiểm thử và In ra kết quả: Kiểm thử mô hình trên tập dữ liệu kiểm thử và in ra các câu, nhãn từ loại thực tế, và nhãn từ loại dự đoán.

In [3]:
import nltk
from nltk.corpus import treebank
from collections import defaultdict, Counter
import numpy as np

# Load the treebank corpus
sentences = treebank.tagged_sents(tagset='universal')

# Split into train and test sets
train_size = int(0.9 * len(sentences))
train_sents = sentences[:train_size]
test_sents = sentences[train_size:]



In [4]:
# States (POS tags)
states = set(tag for sent in train_sents for word, tag in sent)

# Initialize probabilities
start_p = defaultdict(float)
trans_p = defaultdict(lambda: defaultdict(float))
emit_p = defaultdict(lambda: defaultdict(float))

# Start probability
start_tag_counts = Counter(sent[0][1] for sent in train_sents)
total_start_tags = sum(start_tag_counts.values())
for tag, count in start_tag_counts.items():
    start_p[tag] = count / total_start_tags

# Transition probabilities
transition_counts = defaultdict(Counter)
for sent in train_sents:
    for i in range(1, len(sent)):
        prev_tag = sent[i-1][1]
        curr_tag = sent[i][1]
        transition_counts[prev_tag][curr_tag] += 1

for prev_tag, curr_tag_counts in transition_counts.items():
    total_transitions = sum(curr_tag_counts.values())
    for curr_tag, count in curr_tag_counts.items():
        trans_p[prev_tag][curr_tag] = count / total_transitions

# Emission probabilities
emission_counts = defaultdict(Counter)
word_counts = Counter()
for sent in train_sents:
    for word, tag in sent:
        emission_counts[tag][word] += 1
        word_counts[word] += 1

for tag, word_counts in emission_counts.items():
    total_emissions = sum(word_counts.values())
    for word, count in word_counts.items():
        emit_p[tag][word] = count / total_emissions


In [7]:
print("Start Probabilities:")
for tag, prob in start_p.items():
    print(f"{tag}: {prob:.4f}")
print()

Start Probabilities:
NOUN: 0.2811
DET: 0.2365
ADP: 0.1303
PRON: 0.0769
.: 0.0880
NUM: 0.0080
ADV: 0.0520
ADJ: 0.0403
CONJ: 0.0511
X: 0.0247
VERB: 0.0097
PRT: 0.0014



In [8]:
print("Transition Probabilities:")
for prev_tag, curr_tag_counts in trans_p.items():
    for curr_tag, prob in curr_tag_counts.items():
        print(f"{prev_tag} -> {curr_tag}: {prob:.4f}")
print()

Transition Probabilities:
NOUN -> NOUN: 0.2631
NOUN -> .: 0.2429
NOUN -> ADJ: 0.0119
NOUN -> ADP: 0.1768
NOUN -> NUM: 0.0097
NOUN -> VERB: 0.1461
NOUN -> ADV: 0.0171
NOUN -> PRT: 0.0432
NOUN -> DET: 0.0128
NOUN -> CONJ: 0.0423
NOUN -> PRON: 0.0049
NOUN -> X: 0.0291
. -> NUM: 0.1127
. -> VERB: 0.1299
. -> DET: 0.1426
. -> NOUN: 0.1867
. -> ADP: 0.0723
. -> .: 0.1032
. -> PRON: 0.0637
. -> ADV: 0.0516
. -> ADJ: 0.0450
. -> CONJ: 0.0612
. -> PRT: 0.0027
. -> X: 0.0285
NUM -> NOUN: 0.3541
NUM -> .: 0.1187
NUM -> CONJ: 0.0149
NUM -> PRT: 0.0301
NUM -> NUM: 0.1788
NUM -> X: 0.2054
NUM -> ADP: 0.0370
NUM -> VERB: 0.0193
NUM -> ADJ: 0.0342
NUM -> ADV: 0.0028
NUM -> DET: 0.0035
NUM -> PRON: 0.0013
ADJ -> .: 0.0661
ADJ -> NOUN: 0.6983
ADJ -> CONJ: 0.0170
ADJ -> ADJ: 0.0678
ADJ -> ADP: 0.0758
ADJ -> X: 0.0220
ADJ -> NUM: 0.0211
ADJ -> VERB: 0.0116
ADJ -> ADV: 0.0045
ADJ -> PRT: 0.0107
ADJ -> DET: 0.0045
ADJ -> PRON: 0.0007
VERB -> VERB: 0.1669
VERB -> DET: 0.1341
VERB -> NOUN: 0.1078
VERB -> X: 0

In [9]:
print("Emission Probabilities:")
for tag, word_counts in emit_p.items():
    for word, prob in word_counts.items():
        print(f"{tag} -> {word}: {prob:.4f}")
print()

Streaming output truncated to the last 5000 lines.
ADJ -> unfathomable: 0.0002
ADJ -> implicit: 0.0002
ADJ -> nutty: 0.0002
ADJ -> imaginative: 0.0002
ADJ -> funny: 0.0005
ADJ -> disaffected: 0.0002
ADJ -> hard-drinking: 0.0002
ADJ -> nearly-30: 0.0002
ADJ -> elusive: 0.0002
ADJ -> sinister: 0.0002
ADJ -> erudite: 0.0002
ADJ -> prescient: 0.0002
ADJ -> sassy: 0.0002
ADJ -> docile: 0.0002
ADJ -> solicitous: 0.0002
ADJ -> Christian: 0.0002
ADJ -> sweet: 0.0003
ADJ -> roughhewn: 0.0002
ADJ -> a: 0.0003
ADJ -> 40-year-old: 0.0002
ADJ -> fluent: 0.0002
ADJ -> youthful: 0.0002
ADJ -> idiomatic: 0.0002
ADJ -> Nipponese: 0.0003
ADJ -> fabled: 0.0002
ADJ -> important: 0.0029
ADJ -> humble: 0.0002
ADJ -> uncomplaining: 0.0002
ADJ -> obedient: 0.0002
ADJ -> male: 0.0003
ADJ -> unrecognizable: 0.0003
ADJ -> foul: 0.0003
ADJ -> even: 0.0005
ADJ -> strict: 0.0005
ADJ -> amusing: 0.0003
ADJ -> enormous: 0.0003
ADJ -> worth: 0.0003
ADJ -> petulant: 0.0002
ADJ -> impudent: 0.0002
ADJ -> supercilious: 0

In [5]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    path = {}

    # Initialize base cases (t == 0)
    for state in states:
        V[0][state] = start_p[state] * emit_p[state].get(obs[0], 1e-6)
        path[state] = [state]

    # Run Viterbi for t > 0
    for t in range(1, len(obs)):
        V.append({})
        new_path = {}

        for curr_state in states:
            max_prob, prev_state_selected = max(
                (V[t-1][prev_state] * trans_p[prev_state].get(curr_state, 1e-6) * emit_p[curr_state].get(obs[t], 1e-6), prev_state)
                for prev_state in states
            )
            V[t][curr_state] = max_prob
            new_path[curr_state] = path[prev_state_selected] + [curr_state]

        path = new_path

    # Find the final most probable state and its path
    n = len(obs) - 1
    max_prob, final_state = max((V[n][state], state) for state in states)

    return max_prob, path[final_state]


In [6]:
def test_viterbi(test_sents, states, start_p, trans_p, emit_p):
    correct = 0
    total = 0

    for sent in test_sents:
        obs = [word for word, tag in sent]
        true_tags = [tag for word, tag in sent]

        _, pred_tags = viterbi(obs, states, start_p, trans_p, emit_p)

        correct += sum(p == t for p, t in zip(pred_tags, true_tags))
        total += len(true_tags)

    accuracy = correct / total
    return accuracy

accuracy = test_viterbi(test_sents, states, start_p, trans_p, emit_p)
print(f"Accuracy: {accuracy:.4f}")


Accuracy: 0.9313


In [11]:
# Thêm kiểm thử với ví dụ POS tagging khác
example_sentence = "The quick brown fox jumps over the lazy dog"
example_obs = example_sentence.split()

# Chạy Viterbi trên câu ví dụ
_, example_tags = viterbi(example_obs, states, start_p, trans_p, emit_p)

print(f"Example Sentence: {example_sentence}")
print(f"Predicted Tags: {' '.join(example_tags)}")

Example Sentence: The quick brown fox jumps over the lazy dog
Predicted Tags: DET ADJ NOUN NOUN NOUN ADP DET NOUN NOUN


In [12]:
# Thêm kiểm thử với ví dụ POS tagging khác
example_sentence = "He enjoys playing football on weekends"
example_obs = example_sentence.split()

# Chạy Viterbi trên câu ví dụ
_, example_tags = viterbi(example_obs, states, start_p, trans_p, emit_p)

print(f"Example Sentence: {example_sentence}")
print(f"Predicted Tags: {' '.join(example_tags)}")

Example Sentence: He enjoys playing football on weekends
Predicted Tags: PRON VERB VERB NOUN ADP NOUN
